In [1]:
%%markdown
## Prereq
Run the following in the DB
```
CREATE USER your_user WITH PASSWORD 'YOURPW';

GRANT ALL PRIVILEGES ON DATABASE chat_db TO chat_user;

CREATE TABLE IF NOT EXISTS chat_messages (
    id VARCHAR(64) PRIMARY key,
    chat_messages JSONB NOT NULL
);
```

## Prereq
Run the following in the DB
```
CREATE USER chat_user WITH PASSWORD 'chatuser@1234';

GRANT ALL PRIVILEGES ON DATABASE chat_db TO chat_user;

CREATE TABLE IF NOT EXISTS chat_messages (
    id VARCHAR(64) PRIMARY key,
    chat_messages JSONB NOT NULL
);
```


In [2]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, SystemMessage, HumanMessage, messages_to_dict, BaseMessage, messages_from_dict, trim_messages, messages_from_dict
from typing import Sequence
from dotenv import load_dotenv
import uuid
import json
import psycopg2
from psycopg2.extras import Json

In [3]:
MODEL = "gpt-4o-mini"
DB_HOST = 'localhost'
DB_PORT = '5433'
DB_NAME = 'chat_db'
DB_USER = ''
DB_PASSWORD = ''

load_dotenv()

True

In [4]:
model = ChatOpenAI(model=MODEL)

In [5]:
%%markdown
## DB Functions and connections

## DB Functions and connections


In [6]:
def insert_chat_message(chat_id, chat_dict):
    try:
        # Insert the data into the table
        insert_query = """
            INSERT INTO chat_messages (id, chat_messages)
            VALUES (%s, %s)
        """
        cursor.execute(insert_query, (str(chat_id), Json(chat_dict)))

        # Commit the transaction
        connection.commit()

        print(f"Inserted chat message with UUID: {chat_id}")
    except Exception as e:
        print(f"Error inserting data: {e}")
        connection.rollback()

def fetch_chat_conversation(chat_id):
    """
    Fetches a specific chat conversation by its UUID.

    :param chat_id: UUID of the chat to retrieve.
    :return: List of chat messages if found, else None.
    """
    try:
        # SQL query to select chat_messages for the given id
        select_query = "SELECT chat_messages FROM chat_messages WHERE id = %s"
        cursor.execute(select_query, (str(chat_id),))
        result = cursor.fetchone()

        if result is None:
            print(f"No chat conversation found with UUID: {chat_id}")
            return None

        chat_messages = result[0]

        # Ensure the chat_messages is a list
        if not isinstance(chat_messages, list):
            print(f"Unexpected data format for chat_messages: {type(chat_messages)}")
            return None

        return chat_messages

    except Exception as e:
        print(f"Error fetching chat conversation: {e}")
        return None

def update_chat_messages(chat_id, updated_messages):
    """
    Fetches the existing chat_messages JSON, appends a new message in Python,
    and replaces the existing JSON in the database with the updated version.

    :param chat_id: UUID of the chat to update.
    :param new_message: Dictionary representing the new message to append.
    """
    try:
        update_query = """
            UPDATE chat_messages
            SET chat_messages = %s
            WHERE id = %s
        """
        cursor.execute(update_query, (Json(updated_messages), str(chat_id)))
        connection.commit()
        print(f"Successfully updated new message to chat with UUID: {chat_id}")
    except Exception as e:
        print(f"Error updating chat_messages: {e}")
        connection.rollback()

In [7]:
try:
    connection = psycopg2.connect(
        host=DB_HOST,
        port=DB_PORT,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    )
    cursor = connection.cursor()
    print("Connected to the PostgreSQL database successfully.")
except Exception as e:
    print(f"Error connecting to the database: {e}")
    exit(1)

Connected to the PostgreSQL database successfully.


In [13]:
%%markdown
## List is created on the fly from DB. Excess messages are truncated to stay in the context limit. 

## List is created on the fly from DB. Excess messages are truncated to stay in the context limit. 


In [11]:
messages: Sequence[BaseMessage] = []
system_prompt = SystemMessage(content="Your name is 'Agustrama' and you are a helpful assistant responsible for helping the users in the best possible way. On the first conversation make sure to introduce your self.")
messages.append(system_prompt)

In [12]:
while True:
    # On first time chat start create a DB entry
    if len(messages) == 1:
        # Creating new Chat
        chat_id = uuid.uuid4()
        insert_chat_message(chat_id, messages_to_dict(messages))
    else:
        messages = messages_from_dict(fetch_chat_conversation(chat_id))
    
    msg = input("User: ")
    if msg == 'q':
        end_statement = f"Agustrama: Thank you very much for contacting me. Hope to see you soon."
        msg_dict = messages_to_dict(messages)
        print(end_statement)
        update_chat_messages(chat_id, msg_dict)
        connection.close()
        break;
    
    user_prompt = HumanMessage(content=msg)
    messages.append(user_prompt)
    
    if len(messages) >= 7:
        chat_messages = trim_messages(
            messages,
            token_counter=len,
            max_tokens=5,
            start_on="human",
            end_on=("human", "tool"),
            include_system=True,
            allow_partial=False
        )
    
        model_response = model.invoke(chat_messages)
        print(f"Agustrama: {model_response.content}")
        messages.append(model_response)
        update_chat_messages(chat_id, messages_to_dict(messages))
    else:
        model_response = model.invoke(messages)
        print(f"Agustrama: {model_response.content}")
        messages.append(model_response)
        update_chat_messages(chat_id, messages_to_dict(messages))

Inserted chat message with UUID: 2647b47b-3db5-4eb0-9fd8-6d1640010723


User:  Hi


Agustrama: Hello! I'm Agustrama, your helpful assistant. How can I assist you today?
Successfully updated new message to chat with UUID: 2647b47b-3db5-4eb0-9fd8-6d1640010723


User:  Give me a simple word play joke.


Agustrama: Sure! Here's a classic one for you:

Why did the scarecrow win an award?

Because he was outstanding in his field!
Successfully updated new message to chat with UUID: 2647b47b-3db5-4eb0-9fd8-6d1640010723


User:  Can you provide another one?


Agustrama: Of course! Here's another one for you:

What do you call fake spaghetti?

An impasta!
Successfully updated new message to chat with UUID: 2647b47b-3db5-4eb0-9fd8-6d1640010723


User:  q


Agustrama: Thank you very much for contacting me. Hope to see you soon.
Successfully updated new message to chat with UUID: 2647b47b-3db5-4eb0-9fd8-6d1640010723
